In [2]:
import pandas as pd

inputfile = '../datasets/data.csv'
outputfile = "../datasets/PollutedCities.csv"

df = pd.read_csv(inputfile, encoding = "ISO-8859-1")

df = df.drop(['stn_code', 'sampling_date', 'agency', 'type', 'location_monitoring_station','pm2_5', 'spm'], axis = 1)
years = ['2013','2014']
df = df[df["date"].str.contains('|'.join(years), na=False)]
df.loc[:, 'date'] = df['date'].str.split('-', expand=True)[0]

df['state'] = df['state'].replace(['Uttaranchal'],'Uttarakhand')
df['state'] = df['state'].replace(['Telangana'],'Andhra Pradesh')
df = df.groupby(['state','location', 'date']).mean()

# Only this row has NA in no2 and so2 in 2014, rspm is present for all cities in 2014
df.loc['Andhra Pradesh','Eluru','2014']['so2'] = df.loc['Andhra Pradesh','Eluru','2013']['so2']
df.loc['Andhra Pradesh','Eluru','2014']['no2'] = df.loc['Andhra Pradesh','Eluru','2013']['no2']
df.loc['Andhra Pradesh','Eluru','2014']

df = df.reset_index(level=['state','location','date'])
df = df[df['date'].str.contains('2014', na=False)]
df = df.drop(['date'], axis = 1)

df['Mean-Pollutant-Concentration'] = df.mean(axis=1)
df = df.drop(['so2', 'no2', 'rspm'], axis = 1)
df = df.sort_values(by = ['state','Mean-Pollutant-Concentration'], ascending=[True,False])

df2 = pd.read_csv('../datasets/Zscore_MapFile.csv')
df2 = df2[df2['Year'] == 2014 ]
df2 = df2.drop(['Year', 'zscore', 'spot'], axis = 1)

# Check if inner join is safe (must give empty set)
state = set(df['state'])
state2 = set(df2['State'])
state - state2

df2.rename(columns = {'State':'state', 'Mpc':'state-Mpc'}, inplace = True)

df = pd.merge(df, df2)
df = df.groupby('state').head(5)

df.to_csv(outputfile, index=False)

In [ ]:
import folium
import pandas as pd
import sys
import branca
import numpy as np
from branca.element import *

state_geo = json.load(open('india_state.geojson'))
state_air = pd.read_csv('PollutedCities.csv')

In [ ]:
dic={}
for index,row in state_air.iterrows():
    st=row['state']
    loc=row['location']
    mpc=row['Mean-Pollutant-Concentration']
    if dic.get(st)==None:
        dic[st]=[loc,mpc]
    else:
        dic[st]+=[loc,mpc]

In [ ]:
states=[]
def my_color_function(feature):
    states.append(feature['properties']['NAME_1'])
    if feature['properties']['NAME_1'] not in state_air.values:
        return '#050201' #Black
    elif list(state_air[(state_air['state']==feature['properties']['NAME_1'])]['state-Mpc'])[0]>=65:
        return '#E31111' #Red
    elif list(state_air[(state_air['state']==feature['properties']['NAME_1'])]['state-Mpc'])[0]>=45 and list(state_air[(state_air['state']==feature['properties']['NAME_1'])]['state-Mpc'])[0]<65:
        return '#BDB76B' #Yellow
    else:
        return '#2A610A' #Green
m = folium.Map([21,78], tiles=None, zoom_start=4)

fg=folium.FeatureGroup('Cartodbpositron',show=False)
fg.add_child(folium.TileLayer('cartodbpositron'))
fg2=folium.FeatureGroup('OpenStreetMap',show=False)
fg2.add_child(folium.TileLayer('openstreetmap'))
fg1=folium.FeatureGroup('Terrain')
fg1.add_child(folium.TileLayer(tiles='https://stamen-tiles-{s}.a.ssl.fastly.net/terrain-background/{z}/{x}/{y}{r}.png',attr= 'Map tiles by <a href="http://stamen.com">Stamen Design</a>, <a href="http://creativecommons.org/licenses/by/3.0">CC BY 3.0</a> &mdash; Map data &copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors'))

m.add_child(fg)
m.add_child(fg1)
m.add_child(fg2)
folium.GeoJson(
    state_geo,
    style_function=lambda feature: {
        'fillColor': my_color_function(feature),
        'fillOpacity': 0.7,
        'color': 'black',
        'weight': 1.5,
    }
).add_to(m)
folium.LayerControl().add_to(m)
m

In [ ]:
for state in states:
    if state not in state_air.values:
        continue
    rowcolor=""
    city0=""
    city1=""
    city2=""
    city3=""
    city4=""
    color0='#FFFFFF'
    color1='#FFFFFF'
    color2='#FFFFFF'
    color3='#FFFFFF'
    color4='#FFFFFF'
    mpc0=""
    mpc1=""
    mpc2=""
    mpc3=""
    mpc4=""
    if len(dic[state])==2:
        city0=dic[state][0]
        mpc0=round(dic[state][1],3)
        if(mpc0>=65):
            color0='#E31111'
        elif(mpc0<65 and mpc0>=45):
            color0='#BDB76B'
        else:
            color0='#2A610A'
    elif len(dic[state])==4:
        city0=dic[state][0]
        mpc0=round(dic[state][1],3)
        city1=dic[state][2]
        mpc1=round(dic[state][3],3)
        if(mpc0>=65):
            color0='#E31111'
        elif(mpc0<65 and mpc0>=45):
            color0='#BDB76B'
        else:
            color0='#2A610A'
        if(mpc1>=65):
            color1='#E31111'
        elif(mpc1<65 and mpc1>=45):
            color1='#BDB76B'
        else:
            color1='#2A610A'
    elif len(dic[state])==6:
        city0=dic[state][0]
        mpc0=round(dic[state][1],3)
        city1=dic[state][2]
        mpc1=round(dic[state][3],3)
        city2=dic[state][4]
        mpc2=round(dic[state][5],3)
        if(mpc0>=65):
            color0='#E31111'
        elif(mpc0<65 and mpc0>=45):
            color0='#BDB76B'
        else:
            color0='#2A610A'
        if(mpc1>=65):
            color1='#E31111'
        elif(mpc1<65 and mpc1>=45):
            color1='#BDB76B'
        else:
            color1='#2A610A'
        if(mpc2>=65):
            color2='#E31111'
        elif(mpc2<65 and mpc2>=45):
            color2='#BDB76B'
        else:
            color2='#2A610A'
    elif len(dic[state])==8:
        city0=dic[state][0]
        mpc0=round(dic[state][1],3)
        city1=dic[state][2]
        mpc1=round(dic[state][3],3)
        city2=dic[state][4]
        mpc2=round(dic[state][5],3)
        city3=dic[state][6]
        mpc3=round(dic[state][7],3)
        if(mpc0>=65):
            color0='#E31111'
        elif(mpc0<65 and mpc0>=45):
            color0='#BDB76B'
        else:
            color0='#2A610A'
        if(mpc1>=65):
            color1='#E31111'
        elif(mpc1<65 and mpc1>=45):
            color1='#BDB76B'
        else:
            color1='#2A610A'
        if(mpc2>=65):
            color2='#E31111'
        elif(mpc2<65 and mpc2>=45):
            color2='#BDB76B'
        else:
            color2='#2A610A'
        if(mpc3>=65):
            color3='#E31111'
        elif(mpc3<65 and mpc3>=45):
            color3='#BDB76B'
        else:
            color3='#2A610A'
    elif len(dic[state])==10:
        city0=dic[state][0]
        mpc0=round(dic[state][1],3)
        city1=dic[state][2]
        mpc1=round(dic[state][3],3)
        city2=dic[state][4]
        mpc2=round(dic[state][5],3)
        city3=dic[state][6]
        mpc3=round(dic[state][7],3)
        city4=dic[state][8]
        mpc4=round(dic[state][9],3)
        if(mpc0>=65):
            color0='#E31111'
        elif(mpc0<65 and mpc0>=45):
            color0='#BDB76B'
        else:
            color0='#2A610A'
        if(mpc1>=65):
            color1='#E31111'
        elif(mpc1<65 and mpc1>=45):
            color1='#BDB76B'
        else:
            color1='#2A610A'
        if(mpc2>=65):
            color2='#E31111'
        elif(mpc2<65 and mpc2>=45):
            color2='#BDB76B'
        else:
            color2='#2A610A'
        if(mpc3>=65):
            color3='#E31111'
        elif(mpc3<65 and mpc3>=45):
            color3='#BDB76B'
        else:
            color3='#2A610A'
        if(mpc4>=65):
            color4='#E31111'
        elif(mpc4<65 and mpc4>=45):
            color4='#BDB76B'
        else:
            color4='#2A610A'
    coordinates=[str(list(state_air[(state_air['state']==state)]['xco'])[0]),str(list(state_air[(state_air['state']==state)]['yco'])[0])]
    html="""<!DOCTYPE html>
<html>
<head>
<script>
</script>
		<title>"""+state+""" </title>
		<style>
		table, td, th {
			border: 1px solid black;
		}
    axisX:{
            title:"Year",
			labelFormatter: function(e){
				return  e.value;
			}
		},
    axisY:{
        title:"MPC",
    },
		table {
			border-collapse: collapse;
		width: 100%;
        margin-top:0;
		}

		th {
			height: 70px;
		}
        h1 {
	margin-bottom:0;
    margin:0;
	}
		</style>
</head>
<body>
<h1 style="text-align:center">"""+state+"""</h1>
<h2 style="text-align:center">"""+"Most polluted cities (2014)"+"""</h1>
    <style> table.example-table, .example-table td {   border: 1px solid black;   border-collapse: collapse; } </style> 
	<table class="example-table">
	<tr>
    <th style="background-color: """+ "#2A799C" +""";">SNo</th>
	<th style="background-color: """+ "#2A799C" +""";">City</th>
	<th style="background-color: """+ "#2A799C" +""";">Mean Pollutant concentration</th>
	</tr>
    <tr>
    <td style="background-color: """+ color0 +""";">"""+"1"+"""</td>
    <td style="background-color: """+ color0 +""";">"""+str(city0)+"""</td>
    <td style="background-color: """+ color0 +""";">"""+str(mpc0)+"""</td>
    </tr>
    <td style="background-color: """+ color1 +""";">"""+"2"+"""</td>
    <td style="background-color: """+ color1 +""";">"""+str(city1)+"""</td>
    <td style="background-color: """+ color1 +""";">"""+str(mpc1)+"""</td>
    </tr>
    <td style="background-color: """+ color2 +""";">"""+"3"+"""</td>
    <td style="background-color: """+ color2 +""";">"""+str(city2)+"""</td>
    <td style="background-color: """+ color2 +""";">"""+str(mpc2)+"""</td>
    </tr>
    <td style="background-color: """+ color3 +""";">"""+"4"+"""</td>
    <td style="background-color: """+ color3 +""";">"""+str(city3)+"""</td>
    <td style="background-color: """+ color3 +""";">"""+str(mpc3)+"""</td>
    </tr>
    <td style="background-color: """+ color4 +""";">"""+"5"+"""</td>
    <td style="background-color: """+ color4 +""";">"""+str(city4)+"""</td>
    <td style="background-color: """+ color4 +""";">"""+str(mpc4)+"""</td>
    
</table>
</body>
</html>
"""
    iframe = branca.element.IFrame(html=html,width=320,height=250)
    popup = folium.Popup(iframe,parse_html=True)
    folium.Marker(coordinates,popup=popup,icon=folium.features.CustomIcon('google1.png',icon_size=(13,13))).add_to(m)
#Adjust marker type for different spots


In [ ]:
from branca.element import Template, MacroElement

template = """
{% macro html(this, kwargs) %}

<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <title>Pollution-map</title>
  <link rel="stylesheet" href="//code.jquery.com/ui/1.12.1/themes/base/jquery-ui.css">

  <script src="https://code.jquery.com/jquery-1.12.4.js"></script>
  <script src="https://code.jquery.com/ui/1.12.1/jquery-ui.js"></script>
  
  <script>
  $( function() {
    $( "#maplegend" ).draggable({
                    start: function (event, ui) {
                        $(this).css({
                            right: "auto",
                            top: "auto",
                            bottom: "auto"
                        });
                    }
                });
});

  </script>
</head>
<body>

 
<div id='maplegend' class='maplegend' 
    style='position: absolute; z-index:9999; border:2px solid grey; background-color:rgba(255, 255, 255, 0.8);
     border-radius:6px; padding: 10px; font-size:14px; right: 20px; bottom: 20px;'>
     
<div class='legend-title'>Legend</div>
<div class='legend-scale'>
  <ul class='legend-labels'>
    <li><span style='background:red;opacity:0.7;'></span>MPC>=65</li>
    <li><span style='background:#BDB76B;opacity:0.7;'></span>45>=MPC>65</li>
    <li><span style='background:green;opacity:0.7;'></span>MPC<45</li>
    <li><span style='background:black;opacity:0.7;'></span>Insufficient Data</li>
    
  </ul>
</div>
</div>
 
</body>
</html>

<style type='text/css'>
  .maplegend .legend-title {
    text-align: left;
    margin-bottom: 5px;
    font-weight: bold;
    font-size: 90%;
    }
  .maplegend .legend-scale ul {
    margin: 0;
    margin-bottom: 5px;
    padding: 0;
    float: left;
    list-style: none;
    }
  .maplegend .legend-scale ul li {
    font-size: 80%;
    list-style: none;
    margin-left: 0;
    line-height: 18px;
    margin-bottom: 2px;
    }
  .maplegend ul.legend-labels li span {
    display: block;
    float: left;
    height: 16px;
    width: 30px;
    margin-right: 5px;
    margin-left: 0;
    border: 1px solid #999;
    }
  .maplegend .legend-source {
    font-size: 80%;
    color: #777;
    clear: both;
    }
  .maplegend a {
    color: #777;
    }
</style>
{% endmacro %}"""

macro = MacroElement()
macro._template = Template(template)

m.get_root().add_child(macro)
m.save('2014mpc-map.html')
m